In [41]:
from io import BytesIO
import IPython
import json
import os
from PIL import Image
import requests
import time

In [42]:
#@title Connect to the Stability API

import getpass
# @markdown To get your API key visit https://platform.stability.ai/account/keys
STABILITY_KEY = 'sk-fpFvSMT2n7dfLe1rWrAL6W6NQ9JsTWZGK63CoTdJM94IfcC0'


In [43]:
#@title Define functions

def send_generation_request(
    host,
    params,
    files = None
):
    headers = {
        "Accept": "image/*",
        "Authorization": f"Bearer {STABILITY_KEY}"
    }

    if files is None:
        files = {}

    # Encode parameters
    image = params.pop("image", None)
    mask = params.pop("mask", None)
    if image is not None and image != '':
        files["image"] = open(image, 'rb')
    if mask is not None and mask != '':
        files["mask"] = open(mask, 'rb')
    if len(files)==0:
        files["none"] = ''

    # Send request
    print(f"Sending REST request to {host}...")
    response = requests.post(
        host,
        headers=headers,
        files=files,
        data=params
    )
    if not response.ok:
        raise Exception(f"HTTP {response.status_code}: {response.text}")

    return response

def send_async_generation_request(
    host,
    params,
    files = None
):
    headers = {
        "Accept": "application/json",
        "Authorization": f"Bearer {STABILITY_KEY}"
    }

    if files is None:
        files = {}

    # Encode parameters
    image = params.pop("image", None)
    mask = params.pop("mask", None)
    if image is not None and image != '':
        files["image"] = open(image, 'rb')
    if mask is not None and mask != '':
        files["mask"] = open(mask, 'rb')
    if len(files)==0:
        files["none"] = ''

    # Send request
    print(f"Sending REST request to {host}...")
    response = requests.post(
        host,
        headers=headers,
        files=files,
        data=params
    )
    if not response.ok:
        raise Exception(f"HTTP {response.status_code}: {response.text}")

    # Process async response
    response_dict = json.loads(response.text)
    generation_id = response_dict.get("id", None)
    assert generation_id is not None, "Expected id in response"

    # Loop until result or timeout
    timeout = int(os.getenv("WORKER_TIMEOUT", 500))
    start = time.time()
    status_code = 202
    while status_code == 202:
        print(f"Polling results at https://api.stability.ai/v2beta/results/{generation_id}")
        response = requests.get(
            f"https://api.stability.ai/v2beta/results/{generation_id}",
            headers={
                **headers,
                "Accept": "*/*"
            },
        )

        if not response.ok:
            raise Exception(f"HTTP {response.status_code}: {response.text}")
        status_code = response.status_code
        time.sleep(10)
        if time.time() - start > timeout:
            raise Exception(f"Timeout after {timeout} seconds")

    return response

In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import requests
import base64
from io import BytesIO
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

# Paramètres par défaut pour la génération d'image
DEFAULT_PARAMS = {
    "negative_prompt": "",
    "aspect_ratio": "3:2",
    "seed": 0,
    "output_format": "jpeg"
}

STABLE_IMAGE_API_URL = "https://api.stability.ai/v2beta/stable-image/generate/ultra"



@app.route('/generate-image', methods=['POST'])
def generate_image():
    # Récupérer le prompt depuis la requête front-end
    data = request.json
    prompt = data.get("prompt", "")

    if not prompt:
        return jsonify({"error": "Le prompt est requis"}), 400

    # Préparer les paramètres pour la requête
    params = {
        "prompt": prompt,
        **DEFAULT_PARAMS
    }

    try:
        # Envoi de la requête à l'API
        response = send_generation_request(STABLE_IMAGE_API_URL, params)
        print("response", response)
        # Décoder la réponse
        output_image = response.content
        finish_reason = response.headers.get("finish-reason")
        seed = response.headers.get("seed")

        # Vérification du filtre NSFW
        if finish_reason == "CONTENT_FILTERED":
            return jsonify({"error": "L'image générée a été filtrée pour contenu NSFW"}), 400

        # Encoder l'image en Base64
        encoded_image = base64.b64encode(output_image).decode('utf-8')

        # Retourner l'image encodée avec le seed utilisé
        return jsonify({
            "image": encoded_image,
            "seed": seed,
            "message": "Image générée avec succès"
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Jan/2025 12:03:35] "OPTIONS /generate-image HTTP/1.1" 200 -


Sending REST request to https://api.stability.ai/v2beta/stable-image/generate/ultra...


127.0.0.1 - - [03/Jan/2025 12:03:39] "POST /generate-image HTTP/1.1" 500 -
